In [ ]:
import serial
import mysql.connector

# MySQL Configuration
mysql_host = 'localhost'
mysql_user = 'Zack'
mysql_password = 'Killer0!'
mysql_db = 'weatherstation'

def read_data_from_serial(ser):
    if ser.in_waiting > 0:
        data = ser.readline().decode('utf-8').strip()
        print("Received:", data)  # Add this line for debugging
        data_list = data.split(",")
        if len(data_list) == 3:
            return {
                "temperature": data_list[0],
                "humidity": data_list[1],
                "pressure": data_list[2]
            }
    return None

def save_to_mysql(data):
    try:
        conn = mysql.connector.connect(
            host=mysql_host,
            user=mysql_user,
            password=mysql_password,
            database=mysql_db
        )
        cursor = conn.cursor()
        cursor.execute("INSERT INTO weather (temperature, humidity, pressure) VALUES (%s, %s, %s)", (data['temperature'], data['humidity'], data['pressure']))
        conn.commit()
        cursor.close()
        conn.close()
        print("Data saved to MySQL!")
    except Exception as e:
        print("Error saving data to MySQL:", e)

def receive_data():
    ser = serial.Serial('/dev/ttyUSB0', 9600)  # Change '/dev/ttyUSB0' to your Arduino's serial port

    while True:
        data = read_data_from_serial(ser)
        if data:
            print("Temperature:", data['temperature'], "°C")
            print("Humidity:", data['humidity'], "%")
            print("Pressure:", data['pressure'], "hPa")
            save_to_mysql(data)

if __name__ == "__main__":
    receive_data()


Available COM Ports:
COM1: Communications Port (COM1)
COM3: Arduino MKR WiFi 1010 (COM3)
COM5: Arduino Uno (COM5)
Enter COM port for Arduino (e.g., COM3): COM3
Received: Temperature = 26.77 °C
Received: Humidity = 74.19 %
Received: Pressure = 100.84 hPa
Received: HTTP response code: 200
Received: Data sent to ThingSpeak successfully!
Received: Temperature = 26.77 °C
Received: Humidity = 74.13 %
Received: Pressure = 100.84 hPa
Received: HTTP response code: 200
Received: Data sent to ThingSpeak successfully!
Received: Temperature = 26.77 °C
Received: Humidity = 74.06 %
Received: Pressure = 100.84 hPa
Received: HTTP response code: 200
Received: Data sent to ThingSpeak successfully!
Received: Temperature = 26.77 °C
Received: Humidity = 73.96 %
Received: Pressure = 100.84 hPa
Received: HTTP response code: 200
Received: Data sent to ThingSpeak successfully!
Received: Temperature = 26.77 °C
Received: Humidity = 73.85 %
Received: Pressure = 100.84 hPa
Received: HTTP response code: 200
Received

Received: Data sent to ThingSpeak successfully!
Received: Temperature = 26.78 °C
Received: Humidity = 73.47 %
Received: Pressure = 100.82 hPa
Received: HTTP response code: 200
Received: Data sent to ThingSpeak successfully!
Received: Temperature = 26.78 °C
Received: Humidity = 73.40 %
Received: Pressure = 100.82 hPa
Received: HTTP response code: 200
Received: Data sent to ThingSpeak successfully!
Received: Temperature = 26.78 °C
Received: Humidity = 73.28 %
Received: Pressure = 100.82 hPa
Received: HTTP response code: 200
Received: Data sent to ThingSpeak successfully!
Received: Temperature = 26.78 °C
Received: Humidity = 73.26 %
Received: Pressure = 100.82 hPa
Received: HTTP response code: 200
Received: Data sent to ThingSpeak successfully!
Received: Temperature = 26.78 °C
Received: Humidity = 73.30 %
Received: Pressure = 100.82 hPa
Received: HTTP response code: 200
Received: Data sent to ThingSpeak successfully!
Received: Temperature = 26.78 °C
Received: Humidity = 73.49 %
Received: 

Received: Pressure = 100.80 hPa
Received: HTTP response code: 200
Received: Data sent to ThingSpeak successfully!
Received: Temperature = 26.82 °C
Received: Humidity = 73.59 %
Received: Pressure = 100.80 hPa
Received: HTTP response code: 200
Received: Data sent to ThingSpeak successfully!
Received: Temperature = 26.82 °C
Received: Humidity = 73.56 %
Received: Pressure = 100.80 hPa
Received: HTTP response code: 200
Received: Data sent to ThingSpeak successfully!
Received: Temperature = 26.82 °C
Received: Humidity = 73.45 %
Received: Pressure = 100.80 hPa
Received: HTTP response code: 200
Received: Data sent to ThingSpeak successfully!
Received: Temperature = 26.82 °C
Received: Humidity = 73.22 %
Received: Pressure = 100.80 hPa
Received: HTTP response code: 200
Received: Data sent to ThingSpeak successfully!
Received: Temperature = 26.83 °C
Received: Humidity = 72.93 %
Received: Pressure = 100.80 hPa
Received: HTTP response code: 200
Received: Data sent to ThingSpeak successfully!
Receiv

Received: Temperature = 26.84 °C
Received: Humidity = 72.28 %
Received: Pressure = 100.79 hPa
Received: HTTP response code: 200
Received: Data sent to ThingSpeak successfully!
Received: Temperature = 26.84 °C
Received: Humidity = 72.33 %
Received: Pressure = 100.79 hPa
Received: HTTP response code: 200
Received: Data sent to ThingSpeak successfully!
Received: Temperature = 26.84 °C
Received: Humidity = 72.19 %
Received: Pressure = 100.79 hPa
Received: HTTP response code: 200
Received: Data sent to ThingSpeak successfully!
Received: Temperature = 26.83 °C
Received: Humidity = 72.31 %
Received: Pressure = 100.79 hPa
Received: HTTP response code: 200
Received: Data sent to ThingSpeak successfully!
Received: Temperature = 26.83 °C
Received: Humidity = 72.30 %
Received: Pressure = 100.79 hPa
Received: HTTP response code: 200
Received: Data sent to ThingSpeak successfully!
Received: Temperature = 26.83 °C
Received: Humidity = 72.45 %
Received: Pressure = 100.78 hPa
Received: HTTP response co

Received: Data sent to ThingSpeak successfully!
Received: Temperature = 26.72 °C
Received: Humidity = 74.14 %
Received: Pressure = 100.78 hPa
Received: HTTP response code: 200
Received: Data sent to ThingSpeak successfully!
Received: Temperature = 26.72 °C
Received: Humidity = 74.03 %
Received: Pressure = 100.78 hPa
Received: HTTP response code: 200
Received: Data sent to ThingSpeak successfully!
Received: Temperature = 26.72 °C
Received: Humidity = 73.96 %
Received: Pressure = 100.78 hPa
Received: HTTP response code: 200
Received: Data sent to ThingSpeak successfully!
Received: Temperature = 26.72 °C
Received: Humidity = 74.00 %
Received: Pressure = 100.78 hPa
Received: HTTP response code: 200
Received: Data sent to ThingSpeak successfully!
Received: Temperature = 26.71 °C
Received: Humidity = 73.91 %
Received: Pressure = 100.78 hPa
Received: HTTP response code: 200
Received: Data sent to ThingSpeak successfully!
Received: Temperature = 26.71 °C
Received: Humidity = 73.84 %
Received: 

...

Received: Pressure = 100.64 hPa
Received: HTTP response code: 200
Received: Data sent to ThingSpeak successfully!
Received: Temperature = 29.34 °C
Received: Humidity = 63.55 %
Received: Pressure = 100.64 hPa
Received: HTTP response code: 200
Received: Data sent to ThingSpeak successfully!
Received: Temperature = 29.34 °C
Received: Humidity = 63.55 %
Received: Pressure = 100.64 hPa
Received: HTTP response code: 200
Received: Data sent to ThingSpeak successfully!
Received: Temperature = 29.34 °C
Received: Humidity = 63.52 %
Received: Pressure = 100.64 hPa
Received: HTTP response code: 200
Received: Data sent to ThingSpeak successfully!
Received: Temperature = 29.34 °C
Received: Humidity = 63.53 %
Received: Pressure = 100.64 hPa
Received: HTTP response code: 200
Received: Data sent to ThingSpeak successfully!
Received: Temperature = 29.34 °C
Received: Humidity = 63.50 %
Received: Pressure = 100.64 hPa
Received: HTTP response code: 200
Received: Data sent to ThingSpeak successfully!
Receiv

Received: Temperature = 29.38 °C
Received: Humidity = 63.45 %
Received: Pressure = 100.62 hPa
Received: HTTP response code: 200
Received: Data sent to ThingSpeak successfully!
Received: Temperature = 29.38 °C
Received: Humidity = 63.47 %
Received: Pressure = 100.62 hPa
Received: HTTP response code: 200
Received: Data sent to ThingSpeak successfully!
Received: Temperature = 29.38 °C
Received: Humidity = 63.51 %
Received: Pressure = 100.62 hPa
Received: HTTP response code: 200
Received: Data sent to ThingSpeak successfully!
Received: Temperature = 29.38 °C
Received: Humidity = 63.52 %
Received: Pressure = 100.63 hPa
Received: HTTP response code: 200
Received: Data sent to ThingSpeak successfully!
Received: Temperature = 29.38 °C
Received: Humidity = 63.54 %
Received: Pressure = 100.62 hPa
Received: HTTP response code: 200
Received: Data sent to ThingSpeak successfully!
Received: Temperature = 29.39 °C
Received: Humidity = 63.47 %
Received: Pressure = 100.62 hPa
Received: HTTP response co

Received: Data sent to ThingSpeak successfully!
Received: Temperature = 29.43 °C
Received: Humidity = 63.48 %
Received: Pressure = 100.60 hPa
Received: HTTP response code: 200
Received: Data sent to ThingSpeak successfully!
Received: Temperature = 29.44 °C
Received: Humidity = 63.48 %
Received: Pressure = 100.60 hPa
Received: HTTP response code: 200
Received: Data sent to ThingSpeak successfully!
Received: Temperature = 29.44 °C
Received: Humidity = 63.50 %
Received: Pressure = 100.60 hPa
Received: HTTP response code: 200
Received: Data sent to ThingSpeak successfully!
Received: Temperature = 29.44 °C
Received: Humidity = 63.48 %
Received: Pressure = 100.60 hPa
Received: HTTP response code: 200
Received: Data sent to ThingSpeak successfully!
Received: Temperature = 29.44 °C
Received: Humidity = 63.50 %
Received: Pressure = 100.60 hPa
Received: HTTP response code: 200
Received: Data sent to ThingSpeak successfully!
Received: Temperature = 29.44 °C
Received: Humidity = 63.51 %
Received: 

Received: Pressure = 100.59 hPa
Received: HTTP response code: 200
Received: Data sent to ThingSpeak successfully!
Received: Temperature = 29.50 °C
Received: Humidity = 63.48 %
Received: Pressure = 100.59 hPa
Received: HTTP response code: 200
Received: Data sent to ThingSpeak successfully!
Received: Temperature = 29.50 °C
Received: Humidity = 63.46 %
Received: Pressure = 100.59 hPa
Received: HTTP response code: 200
Received: Data sent to ThingSpeak successfully!
Received: Temperature = 29.50 °C
Received: Humidity = 63.44 %
Received: Pressure = 100.59 hPa
Received: HTTP response code: 200
Received: Data sent to ThingSpeak successfully!
Received: Temperature = 29.50 °C
Received: Humidity = 63.47 %
Received: Pressure = 100.59 hPa
Received: HTTP response code: 200
Received: Data sent to ThingSpeak successfully!
Received: Temperature = 29.50 °C
Received: Humidity = 63.49 %
Received: Pressure = 100.59 hPa
Received: HTTP response code: 200
Received: Data sent to ThingSpeak successfully!
Receiv

Received: Temperature = 29.56 °C
Received: Humidity = 63.43 %
Received: Pressure = 100.57 hPa
Received: HTTP response code: 200
Received: Data sent to ThingSpeak successfully!
Received: Temperature = 29.56 °C
Received: Humidity = 63.46 %
Received: Pressure = 100.57 hPa
Received: HTTP response code: 200
Received: Data sent to ThingSpeak successfully!
Received: Temperature = 29.56 °C
Received: Humidity = 63.45 %
Received: Pressure = 100.57 hPa
Received: HTTP response code: 200
Received: Data sent to ThingSpeak successfully!
Received: Temperature = 29.56 °C
Received: Humidity = 63.46 %
Received: Pressure = 100.57 hPa
Received: HTTP response code: 200
Received: Data sent to ThingSpeak successfully!
Received: Temperature = 29.56 °C
Received: Humidity = 63.46 %
Received: Pressure = 100.57 hPa
Received: HTTP response code: 200
Received: Data sent to ThingSpeak successfully!
Received: Temperature = 29.56 °C
Received: Humidity = 63.49 %
Received: Pressure = 100.57 hPa
Received: HTTP response co